In [1]:
%pip install networkx matplotlib seaborn pulp
from pulp import LpProblem
import pandas as pd    
import matplotlib.pyplot as plt  
import seaborn as sns
from pulp import *
import pickle
import warnings
warnings.filterwarnings("ignore")
from datetime import datetime, timedelta
import random
import csv
import plotly.express


Note: you may need to restart the kernel to use updated packages.


In [2]:
#import Data of Arrival flight 
flights = pd.read_excel("Radarfilght_Tuesday_Dec_19.xlsx",)
flights.insert(0, 'Flights_no', range(1, 1 + len(flights)))
flights

,Flights_no,Arrival_time,Flight_name,From,Airline,Aircraft,ICAO code,plane_size,type
0,1,03:00:00,VZ3579,Zhengzhou (CGO),Thai VietJet Air,320,C,3,International
1,2,04:50:00,3U3767,Chengdu (CTU),Sichuan Airlines,320,C,3,International
2,3,06:50:00,SU634,Krasnoyarsk (KJA),Aeroflot,77W,E,5,International
3,4,06:55:00,DD3141,Chengdu (TFU),Nok Air,738,C,3,International
4,5,07:10:00,EY430,Abu Dhabi (AUH),Etihad Airways,B789 (A6-BLS),C,3,International
...,...,...,...,...,...,...,...,...,...
137,138,23:05:00,MU2813,Nanjing (NKG),China Eastern Airlines,A20N (B-306F),C,3,International
138,139,23:15:00,VZ316,Bangkok (BKK),Thai VietJet Air,320,C,3,Domestic
139,140,23:25:00,FD3023,Bangkok (DMK),Thai AirAsia,320,C,3,Domestic
140,141,23:30:00,FD525,Hong Kong (HKG),Thai AirAsia,320,C,3,International


In [3]:
#import Data of gate allocation rule
airport = pd.read_excel("Table_gate.xlsx",)
airport

,Apron,gate,ICAO code,max_size,flight_type,contact_gate
0,B,1,D,4,Domestic,N
1,B,2,D,4,Domestic,N
2,B,3,D,4,Domestic,N
3,B,4,D,4,Domestic,Y
4,B,5,D,4,Domestic,Y
5,A,6,E,5,Domestic,Y
6,A,7,E,5,Domestic,Y
7,A,8,E,5,Domestic,Y
8,A,9,E,5,Domestic,Y
9,A,10,E,5,Domestic,Y


In [4]:
#list ของ flight
flights_list = flights.Flights_no.values 
print("flights to allocate: ", flights_list)
print("num of flight: ",len(flights_list)) #จำนวน flight ทั้งหมดที่จัดตาม Data เปลี่ยนไปตาม input_data ที่ใส่

flights to allocate:  [  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142]
num of flight:  142


In [5]:
#list ของ gate ที่จะนำมาจัด (ไม่เอาลานจอดเครื่องบินเล็ก 21-28 และหลุมจอดฉุกเฉิน 99)
gate_list = airport.gate.values
print("Available gates: ", gate_list)

#list ของ contact gate
contactgate = airport.loc[airport['contact_gate'] == 'Y']
contactgate_list = contactgate.gate.values
print("Contact gates: ", contactgate_list)


Available gates:  ['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '11' '12R' '12' '12L' '14R' '14'
 '14L' '15' '16' 21 22 23 24 25 26 27 28 '31' '32R' '32' '32L' '33R' '33'
 '33L' '34R' '34' '34L' '35' '36' '37' '38' '39' '40' '99']
Contact gates:  ['4' '5' '6' '7' '8' '9' '10' '11' '12R' '12' '12L' '14R' '14' '14L' '15']


In [6]:
#block นี้ไว้ใส่พวก constrain 
#นำ flight มาใส่ constrain ที่เกี่ยวกับ size และ ประเภท DO/IN แล้วทำ Dic เพื่อนำไปสร้าง chromosome
#เหลือใส่เรื่องเวลา

compatible_gates = {}
for idx, row in flights.iterrows():

    size_condition = airport.max_size >= row.plane_size # 1.flight ใดๆลง gate ไหนได้บ้างตามขนาด wingspan
    
    flighttype_condition = airport.flight_type.str.contains(row.type) # 2.แบ่ง Domestic กับ International

    combined_condition = size_condition & flighttype_condition

    gates_lst = airport[combined_condition].gate.values
    compatible_gates[row.Flights_no] = gates_lst
    
print("Compatible gates for each flights") # Gate ที่สามารถลงจอดได้สำหรับแต่ละ flight
for k, v in compatible_gates.items():
    print(k, v)

Compatible gates for each flights
1 ['11' '12R' '12' '12L' '14R' '14' '14L' '15' '16' '31' '32R' '32' '32L'
 '33R' '33' '33L' '34R' '34' '34L' '35' '36' '37' '38' '39' '40']
2 ['11' '12R' '12' '12L' '14R' '14' '14L' '15' '16' '31' '32R' '32' '32L'
 '33R' '33' '33L' '34R' '34' '34L' '35' '36' '37' '38' '39' '40']
3 ['11' '12' '14' '31' '32' '33' '34']
4 ['11' '12R' '12' '12L' '14R' '14' '14L' '15' '16' '31' '32R' '32' '32L'
 '33R' '33' '33L' '34R' '34' '34L' '35' '36' '37' '38' '39' '40']
5 ['11' '12R' '12' '12L' '14R' '14' '14L' '15' '16' '31' '32R' '32' '32L'
 '33R' '33' '33L' '34R' '34' '34L' '35' '36' '37' '38' '39' '40']
6 ['11' '12R' '12' '12L' '14R' '14' '14L' '15' '16' '31' '32R' '32' '32L'
 '33R' '33' '33L' '34R' '34' '34L' '35' '36' '37' '38' '39' '40']
7 ['11' '12R' '12' '12L' '14R' '14' '14L' '15' '16' '31' '32R' '32' '32L'
 '33R' '33' '33L' '34R' '34' '34L' '35' '36' '37' '38' '39' '40']
8 ['1' '2' '3' '4' '5' '6' '7' '8' '9' '10' '31' '32R' '32' '32L' '33R' '33'
 '33L' '34

In [7]:
#เริ่มทำ Genetic 
import numpy as np
import random 

class Individual():
 def __init__(self,generation=0):
    self.score_evaluation = 0
    self.generation = generation
    #สร้างโครโมโซมโดย index ของ chromosome เป็น flight และ element ใน chromosome เป็น gate
    self.chromosome = [random.choice(i) for key, i in compatible_gates.items()]


 def fitness(self):
    score = 0
    for i in self.chromosome:
        if i in contactgate_list: #objective funtion maximize using contactgate
            score += 1
    self.score_evaluation = score
      
 def crossover(self, other_individual): #สร้าง funtion crossover แบบ 1-point (ตัด 1 จุดแล้วสลับพ่อ-แม่) เดียวค่อยทำ 2-point
    cutoff = round(random.random() * len(self.chromosome))
    print("Position of cutoff",cutoff)

    child1 = np.concatenate((self.chromosome[:cutoff], other_individual.chromosome[cutoff:]))
    child2 = np.concatenate((other_individual.chromosome[:cutoff], self.chromosome[cutoff:]))

    children = [Individual(self.generation + 1), Individual(self.generation + 1)]
    children[0].chromosome = child1
    children[1].chromosome = child2

    return children

 def mutation(self)


In [8]:
individual1 = Individual()
individual1.fitness()

In [9]:
print('Chromosome: ', individual1.chromosome)
print('Score: ', individual1.score_evaluation)

Chromosome:  ['32R', '12', '32', '33', '14L', '34L', '16', '33L', '33R', '14', '32R', '14', '6', '37', '7', '14R', '10', '7', '9', '31', '9', '7', '32', '9', '32R', '1', '14', '32L', '36', '11', '39', '12L', '33', '32', '32', '11', '33R', '36', '31', '31', '4', '32L', '37', '4', '33', '12', '5', '12R', '34L', '32', '10', '14L', '34', '12', '2', '31', '40', '6', '11', '34R', '1', '32L', '34', '3', '8', '32L', '12R', '33', '9', '40', '35', '32R', '7', '37', '11', '31', '35', '32R', '38', '31', '1', '12L', '3', '2', '39', '10', '32', '38', '34', '34R', '32L', '1', '32', '33', '12R', '12R', '33L', '38', '34R', '14', '8', '38', '15', '1', '31', '33R', '31', '4', '14', '31', '33L', '14', '36', '32L', '38', '33L', '40', '14', '36', '1', '34', '36', '38', '32L', '33', '34L', '32R', '31', '12', '12L', '31', '38', '6', '34', '33L', '32L', '32L', '31', '32L', '33R', '36', '33R']
Score:  46


In [10]:
individual2 = Individual()
individual2.fitness()

In [11]:
print('Chromosome: ', individual2.chromosome)
print('Score: ', individual2.score_evaluation)

Chromosome:  ['33R', '14', '31', '33L', '32R', '14R', '16', '4', '32L', '33', '32R', '32', '2', '38', '6', '12R', '8', '10', '38', '39', '33', '7', '34', '1', '33L', '37', '14L', '12', '12R', '33', '32R', '34', '11', '8', '38', '11', '39', '8', '3', '11', '33', '3', '32R', '34R', '32', '31', '34R', '14R', '36', '32', '3', '35', '9', '12L', '37', '32R', '33L', '33R', '31', '34R', '32R', '39', '33', '34L', '9', '32L', '31', '10', '31', '31', '32L', '14R', '34L', '40', '11', '34', '3', '36', '10', '34', '2', '11', '33L', '7', '2', '32R', '14', '36', '4', '5', '14L', '34L', '14', '38', '15', '14', '33', '38', '37', '11', '10', '16', '14L', '7', '34', '33', '6', '2', '12', '5', '37', '36', '34', '36', '32L', '1', '38', '11', '36', '5', '34', '9', '10', '1', '33', '3', '10', '11', '12L', '32', '15', '35', '33R', '11', '34', '14R', '33L', '36', '35', '40', '14R', '32']
Score:  51


In [12]:
children = individual1.crossover(individual2)

Position of cutoff 28


In [15]:
children[0].fitness()
print('Chromosome: ',children[0].chromosome)
print('Score: ',children[0].score_evaluation)

Chromosome:  ['32R' '12' '32' '33' '14L' '34L' '16' '33L' '33R' '14' '32R' '14' '6'
 '37' '7' '14R' '10' '7' '9' '31' '9' '7' '32' '9' '32R' '1' '14' '32L'
 '12R' '33' '32R' '34' '11' '8' '38' '11' '39' '8' '3' '11' '33' '3' '32R'
 '34R' '32' '31' '34R' '14R' '36' '32' '3' '35' '9' '12L' '37' '32R' '33L'
 '33R' '31' '34R' '32R' '39' '33' '34L' '9' '32L' '31' '10' '31' '31'
 '32L' '14R' '34L' '40' '11' '34' '3' '36' '10' '34' '2' '11' '33L' '7'
 '2' '32R' '14' '36' '4' '5' '14L' '34L' '14' '38' '15' '14' '33' '38'
 '37' '11' '10' '16' '14L' '7' '34' '33' '6' '2' '12' '5' '37' '36' '34'
 '36' '32L' '1' '38' '11' '36' '5' '34' '9' '10' '1' '33' '3' '10' '11'
 '12L' '32' '15' '35' '33R' '11' '34' '14R' '33L' '36' '35' '40' '14R'
 '32']
Score:  55


In [16]:
children[1].fitness()
print('Chromosome: ',children[1].chromosome)
print('Score: ',children[1].score_evaluation)


Chromosome:  ['33R' '14' '31' '33L' '32R' '14R' '16' '4' '32L' '33' '32R' '32' '2' '38'
 '6' '12R' '8' '10' '38' '39' '33' '7' '34' '1' '33L' '37' '14L' '12' '36'
 '11' '39' '12L' '33' '32' '32' '11' '33R' '36' '31' '31' '4' '32L' '37'
 '4' '33' '12' '5' '12R' '34L' '32' '10' '14L' '34' '12' '2' '31' '40' '6'
 '11' '34R' '1' '32L' '34' '3' '8' '32L' '12R' '33' '9' '40' '35' '32R'
 '7' '37' '11' '31' '35' '32R' '38' '31' '1' '12L' '3' '2' '39' '10' '32'
 '38' '34' '34R' '32L' '1' '32' '33' '12R' '12R' '33L' '38' '34R' '14' '8'
 '38' '15' '1' '31' '33R' '31' '4' '14' '31' '33L' '14' '36' '32L' '38'
 '33L' '40' '14' '36' '1' '34' '36' '38' '32L' '33' '34L' '32R' '31' '12'
 '12L' '31' '38' '6' '34' '33L' '32L' '32L' '31' '32L' '33R' '36' '33R']
Score:  42


In [ ]:
#ทำลูป genetic 
